# Bayesian optimisation tutorial

*Dependencies:*
- *Plotly*
- *BoTorch*
- *NumPy*

### Setup

In [1]:
import plotly.graph_objects as go
import torch
import plotly

n_observations = 3

confidence_interval_color = "rgba(55, 126, 284, 0.1)"
mean_color = plotly.colors.DEFAULT_PLOTLY_COLORS[0]
observations_color = "rgba(228, 26, 28, 1)"

Define the function we want to maximise:

In [2]:
import numpy as np

def f(x):
    coefficients = np.array([ 8.46958953e-04, -2.86719022e-02,  4.44421240e-01, -4.22183149e+00,
        2.77143526e+01, -1.33390584e+02,  4.81173144e+02, -1.29243034e+03,
        2.51141580e+03, -3.35873651e+03,  2.84604441e+03, -1.28836824e+03,
        1.49286518e+02,  7.94924940e+01, -2.10446769e+01,  6.97397114e+00])
    
    return torch.tensor(np.polyval(coefficients, x))

In [3]:
figure = go.Figure(layout_template="simple_white")

x_bounds = torch.tensor([[0], [5]], dtype=torch.double)
x_eval = torch.linspace(x_bounds[0].detach().cpu().numpy()[0], x_bounds[1].detach().cpu().numpy()[0], int(1e3), dtype=torch.double)

figure.add_trace(
    go.Scatter(
        x=x_eval,
        y=f(x_eval),
        name="True function",
        line_dash="dash",
        line_color="black",
    )
)
figure.update_layout(
    xaxis_title="Input value (x)",
    yaxis_title="Objective value (ϕ)",
    showlegend=True,
    xaxis_range=[x_bounds[0].item(), x_bounds[1].item()],
    yaxis_range=[0, 14],
    legend_orientation="h",
    legend_x=1,
    legend_y=1,
    legend_xanchor="right",
    legend_yanchor="bottom",
    margin=dict(b=0, l=0, r=0, t=0),
    font_size=16,
)
figure.write_image("../images/true_function.svg",  width=800, height=400)

### Initial random sampling

Use **Sobol sampling** to generate low-discrepancy ('well-distributed') initial samples. This isn't particularly important in low-dimensional spaces, but it becomes important as the number of samples and the number of dimensions increases.

In [4]:
from botorch.utils.sampling import draw_sobol_samples
from torch.distributions import Uniform
from plotly.subplots import make_subplots


_ = torch.manual_seed(0)
n_samples = 50
bounds = torch.tensor([[0, 0], [1, 1]], dtype=torch.double)

sobol = draw_sobol_samples(bounds, n=n_samples, q=1).squeeze()
uniform = Uniform(bounds[0], bounds[1]).sample((n_samples,))

sampling_figure = make_subplots(rows=1, cols=2, subplot_titles=["Uniform sampling", "Sobol sampling"], horizontal_spacing=0.15)
sampling_figure.add_traces(
    [
        go.Scatter(
            x=uniform[:, 0],
            y=uniform[:, 1],
            mode="markers",
        ),
        go.Scatter(
            x=sobol[:, 0],
            y=sobol[:, 1],
            mode="markers",
        )
    ],
    rows=[1, 1],
    cols=[1, 2]
)
sampling_figure.update_layout(
    template="simple_white",
    showlegend=False,
    margin=dict(b=0, l=0, r=0, t=40),
    font_size=16,
)

for col in [1, 2]:
    sampling_figure.update_xaxes(
        title=r"$x_1$",
        row=1,
        col=col,
    )
    sampling_figure.update_yaxes(
        title=r"$x_2$",
        row=1,
        col=col,
        title_standoff=1 if col == 2 else None,
    )
sampling_figure.show()
sampling_figure.write_image("../images/sobol_vs_uniform.svg", width=800, height=400)

In [5]:
from botorch.utils.sampling import draw_sobol_samples

_ = torch.manual_seed(1)
observed_xs =  draw_sobol_samples(
        x_bounds, n=1, q=n_observations
        ).squeeze(0)

figure.add_trace(
    go.Scatter(
        x=observed_xs.squeeze(),
        y=f(observed_xs.squeeze()),
        mode="markers",
        name="Observations",
        marker_color=observations_color,
    )
)
figure.write_image("../images/initial_samples.svg", width=800, height=400)

### Fit a GP model

In [6]:
from botorch.models import FixedNoiseGP
from botorch import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood

bo_model = FixedNoiseGP(
                observed_xs,
                f(observed_xs),
                torch.full_like(observed_xs, 1e-3),
                
            )
mll = ExactMarginalLogLikelihood(bo_model.likelihood, bo_model)
fit_gpytorch_mll(mll)
posterior = bo_model(x_eval.unsqueeze(-1))
lower, upper = posterior.confidence_region()
mean = posterior.mean

figure.add_traces(
    [
        go.Scatter(
            x=x_eval,
            y=lower.detach().squeeze(),
            mode="lines",
            line_color=confidence_interval_color,      
            fillcolor=confidence_interval_color,
            showlegend=False,
            legendgroup="confidence_interval",
            name="95% confidence interval "
        ),
        go.Scatter(
            x=x_eval,
            y=upper.detach().squeeze(),
            mode="lines",
            line_color=confidence_interval_color,
            fillcolor=confidence_interval_color,
            fill="tonexty",
            legendgroup="confidence_interval",
            name="95% confidence interval"
        ),
        go.Scatter(
            x=x_eval,
            y=posterior.mean.detach().squeeze(),
            line_color=mean_color,
            name="Posterior mean",
        )
    ]
)
figure.write_image("../images/model_1.svg", width=800, height=400)

### Optimise the acquisition function

The acquisition function uses the model to determine which points to select next.

In [7]:
from botorch.acquisition import qNoisyExpectedImprovement
from botorch.sampling.normal import SobolQMCNormalSampler

acquisition_function = qNoisyExpectedImprovement(
    model=bo_model,
    X_baseline=observed_xs,
    sampler=SobolQMCNormalSampler(sample_shape=torch.Size([256])),
)

acqf_figure = go.Figure(layout_template="simple_white")
acqf_figure.add_trace(
    go.Scatter(
        x=x_eval,
        y=acquisition_function(x_eval.unsqueeze(-1).unsqueeze(-1)).detach(),
        line_color=plotly.colors.DEFAULT_PLOTLY_COLORS[1],
        name="Acquisition function",
    )
)
acqf_figure.update_layout(
    yaxis_title="Expected improvement",
    yaxis_range=[0, 0.4],
    xaxis_title="Input (x)",
    margin=dict(b=0, l=0, r=0, t=0),
    font_size=16,
    legend_orientation="h",
    legend_x=1,
    legend_y=1,
    legend_xanchor="right",
    legend_yanchor="bottom",
)

Performing joint optimisation results in a set of points that *jointly* maximise the acquisition function.
Note that this can result in non-intuitive sets (e.g. splitting either side of a peak).

In [8]:
from botorch.optim import optimize_acqf

candidates, _ = optimize_acqf(
        acq_function=acquisition_function,
        bounds=x_bounds,
        q=n_observations,
        num_restarts=5,
        raw_samples=256,
    )

for i, x in enumerate(candidates):
    candidate_trace = go.Scatter(
            x=torch.ones(2)*x,
            y=[-1e3, 1e3],
            mode="lines",
            line_width=1,
            line_color=observations_color,
            name=f"Next candidates {i}",
            showlegend=False,
        )
    figure.add_trace(candidate_trace)
    acqf_figure.add_trace(candidate_trace)

acqf_figure.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        mode="lines",
        line_color=observations_color,
        name="Next candidates",
        showlegend=True,
    )
)
figure.show()
acqf_figure.show()
figure.write_image("../images/model_1_candidates.svg", width=800, height=400)
acqf_figure.write_image("../images/acqf_1_candidates.svg", width=800, height=400)

### Observe new values

Observe the value of the target function at the candidate locations, then update the model.

In [9]:
observed_xs = torch.cat([observed_xs, candidates])

bo_model = FixedNoiseGP(
                observed_xs,
                f(observed_xs),
                torch.full_like(observed_xs, 1e-3),
            )
mll = ExactMarginalLogLikelihood(bo_model.likelihood, bo_model)
fit_gpytorch_mll(mll)
posterior = bo_model(x_eval.unsqueeze(-1))
lower, upper = posterior.confidence_region()
mean = posterior.mean

figure.update_traces(
    y=lower.detach().squeeze(),
    selector=dict(name="95% confidence interval ")
)

figure.update_traces(
    y=upper.detach().squeeze(),
    selector=dict(name="95% confidence interval")
)

figure.update_traces(
    y=posterior.mean.detach().squeeze(),
    selector=dict(name="Posterior mean")
)

figure.update_traces(
    x=observed_xs.squeeze(),
    y=f(observed_xs.squeeze()),
    selector=dict(name="Observations")
)

for i in range(len(candidates)):
    figure.update_traces(
        visible=False,
        selector=dict(name=f"Next candidates {i}")
    )
figure.update_traces(
    visible=False,
    selector=dict(name="Next candidates")
)

figure.show()
figure.write_image("../images/model_2.svg", width=800, height=400)

Now we repeat the previous steps:

1. Optimise the acquisition function

In [10]:
acquisition_function = qNoisyExpectedImprovement(
    model=bo_model,
    X_baseline=observed_xs,
    sampler=SobolQMCNormalSampler(sample_shape=torch.Size([256])),
)
acqf_figure.update_traces(
    y=acquisition_function(x_eval.unsqueeze(-1).unsqueeze(-1)).squeeze().detach(),
    selector=dict(name="Acquisition function")
)

candidates, _ = optimize_acqf(
        acq_function=acquisition_function,
        bounds=x_bounds,
        q=n_observations,
        num_restarts=5,
        raw_samples=256,
    )

for i, x in enumerate(candidates):
    figure.update_traces(
        x=torch.ones(2)*x,
        visible=True,
        selector=dict(name=f"Next candidates {i}")
    )
    acqf_figure.update_traces(
        x=torch.ones(2)*x,
        selector=dict(name=f"Next candidates {i}")
    )

figure.show()
acqf_figure.show()
figure.write_image("../images/model_2_candidates.svg", width=800, height=400)
acqf_figure.write_image("../images/acqf_2_candidates.svg", width=800, height=400)

2. Observe new values
3. Update model

In [11]:
observed_xs = torch.cat([observed_xs, candidates])

bo_model = FixedNoiseGP(
                observed_xs,
                f(observed_xs),
                torch.full_like(observed_xs, 1e-3),
            )
mll = ExactMarginalLogLikelihood(bo_model.likelihood, bo_model)
fit_gpytorch_mll(mll)
posterior = bo_model(x_eval.unsqueeze(-1))
lower, upper = posterior.confidence_region()
mean = posterior.mean

figure.update_traces(
    y=lower.detach().squeeze(),
    selector=dict(name="95% confidence interval ")
)

figure.update_traces(
    y=upper.detach().squeeze(),
    selector=dict(name="95% confidence interval")
)

figure.update_traces(
    y=posterior.mean.detach().squeeze(),
    selector=dict(name="Posterior mean")
)

figure.update_traces(
    x=observed_xs.squeeze(),
    y=f(observed_xs.squeeze()),
    selector=dict(name="Observations")
)

for i in range(len(candidates)):
    figure.update_traces(
        visible=False,
        selector=dict(name=f"Next candidates {i}")
    )

figure.show()
figure.write_image("../images/model_3.svg", width=800, height=400)